In [ ]:
import json
import gc

import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
import numpy as np
import pandas as pd
from tqdm import tqdm

import lightgbm as lgbm

In [ ]:
# Constants

# Paths
INPUT_BASE_PATH = "/kaggle/input/"
RAW_DATA_INPUT_PATH = f"{INPUT_BASE_PATH}/m5-forecasting-accuracy"
PROCESSED_DATA_INPUT_PATH = f"{INPUT_BASE_PATH}/m5-acc"
OUTPUT_BASE_BATH = "/kaggle/working"

# Timestamps
MAX_TRAIN_TIMESTAMP = 1941
START_TEST_TIMESTAMP = 1942
START_TEST_WM_YR_WK = 11617

# Data Input

In [ ]:
# Load raw data

# CALENDAR_DATA = pd.read_csv(f"{RAW_DATA_INPUT_PATH}/calendar.csv")
# SELL_PRICES = pd.read_csv(f"{RAW_DATA_INPUT_PATH}/sell_prices.csv")
# SALES_TRAIN_EVALUATION = pd.read_csv(f"{RAW_DATA_INPUT_PATH}/sales_train_evaluation.csv")

SAMPLE_SUBMISSION = pd.read_csv(f"{RAW_DATA_INPUT_PATH}/sample_submission.csv")
SUBMISSION_INDEX = SAMPLE_SUBMISSION.set_index("id").index
VAL_SUBMISSION = SAMPLE_SUBMISSION[SAMPLE_SUBMISSION["id"].str.contains("validation")]
EVAL_SUBMISSION = SAMPLE_SUBMISSION[SAMPLE_SUBMISSION["id"].str.contains("evaluation")]

In [ ]:
# Load preprocessed data
train_data_set = pd.read_parquet(f"{PROCESSED_DATA_INPUT_PATH}/m5-acc-train.parquet")

In [ ]:
# Drop any columns that we dont need for training or analysis
cols_to_drop = ["av_store_dept_sales", "wm_yr_wk"]
try:
    train_data_set = train_data_set.drop(columns=cols_to_drop)
except KeyError:
    print(f"Columns not found in axis. Skipping ...")

_ = gc.collect()

In [ ]:
# Cast to lower resolution types to save memory
def cast_data_types(dataset: pd.DataFrame) -> pd.DataFrame:
    item_category_cols = ["dept_id", "cat_id", "store_id", "state_id"]
    date_category_cols = ["wm_yr_wk", "weekday", "month", "year", "event_name", "event_type", "snap_CA", "snap_TX", "snap_WI"]
    price_category_cols = ["item_on_sale"]
    for category_col in item_category_cols + date_category_cols + price_category_cols:
        try:
            dataset[category_col] = dataset[category_col].astype("category")
        except KeyError:
            print(f"Column {category_col} does not exist. Skipping ...")
    
    int_cols = ["d", "count"]
    for int_col in int_cols:
        try:
            dataset[int_col] = pd.to_numeric(dataset[int_col], downcast="integer")
        except KeyError:
            print(f"Column {int_col} does not exist. Skipping ...")
            
    float16_cols = ["item_id", "sell_price", "weekday_sin", "weekday_cos", "month_sin", "month_cos"]
    for float_col in float16_cols:
        try:
            dataset[float_col] = dataset[float_col].astype(np.float16)
        except KeyError:
            print(f"Column {int_col} does not exist. Skipping ...")
    
    return dataset

train_data_set = cast_data_types(train_data_set)

# LightGBM Model

In [ ]:
CATEGORICAL_FEATURES = [
    "dept_id",
    "cat_id",
    "store_id",
    "state_id",
    "year",
    "event_name",
    "event_type",
    "snap_CA",
    "snap_TX",
    "snap_WI",
    "item_on_sale",
]
CONTINOUS_FEATURES = [
    "item_id",
    "weekday_sin",
    "weekday_cos",
    "month_sin",
    "month_cos",
    "sell_price",
    "sell_price_diff_1",
    "sell_price_diff_2",
    "sell_price_diff_3",
    "sell_price_diff_7",
#     "av_item_state_sell_price",
#     "av_dept_state_sell_price",
    "count_lag_28",
    "count_lag_29",
    "count_lag_30",
    "count_lag_31",
    "av_store_dept_sales_lag_28",
    "av_store_dept_sales_lag_29",
    "av_store_dept_sales_lag_30",
    "av_store_dept_sales_lag_31"
]
FEATURES = CATEGORICAL_FEATURES + CONTINOUS_FEATURES
LABEL = "count"

# Parameters
TRAIN_PARAMETERS = {
    "objective": "tweedie",
    "learning_rate": 0.05,
    "num_leaves": 2 ** 7 - 1,
    "max_bin": 2 ** 7 - 1,
    "metric": "rmse",
    "force_col_wise": True,
}

In [ ]:
# Train model
horizon = 28
start_t = MAX_TRAIN_TIMESTAMP - horizon
end_t = MAX_TRAIN_TIMESTAMP

train_df = train_data_set[train_data_set["d"] < start_t]
valid_df = train_data_set[(train_data_set["d"] >= start_t) & (train_data_set["d"] <= end_t)]
print(f"Train idx (start, end): ({train_df['d'].min()}, {train_df['d'].max()})")
print(f"Valid idx (start, end): ({valid_df['d'].min()}, {valid_df['d'].max()})")
print("============")

X_train, y_train = train_df[FEATURES], train_df[LABEL]
X_valid, y_valid = valid_df[FEATURES], valid_df[LABEL]
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_valid.shape}, y_test shape: {y_valid.shape}")

train_data = lgbm.Dataset(data=X_train, label=y_train)
valid_data = lgbm.Dataset(data=X_valid, label=y_valid)

model = lgbm.train(
    TRAIN_PARAMETERS,
    num_boost_round=30,
    train_set=train_data,
    valid_sets=[valid_data],
    callbacks=[
        lgbm.early_stopping(stopping_rounds=5),
        lgbm.log_evaluation(period=10),
    ]
)

In [ ]:
# Forecast and save to onto valid df
y_hat = model.predict(data=X_valid)
y_hat = np.clip(y_hat, a_min=0, a_max=np.inf)
y_hat = y_hat.astype(np.int32)

valid_df.loc[:, "forecast"] = y_hat

## LightGBM Model

In [ ]:
def item_level_validation_plot(
    valid_df: pd.DataFrame,
    item_id: int,
):
    """
    Plots forecast and ground truth sales for a given item id across all stores.
    
    :param valid_df: DataFrame containing forecast and ground truth sales data
        by item and store id.
    :param item_id: The item id to plot sales data for.
    """
    valid_for_item = valid_df[valid_df["item_id"] == item_id]
    store_ids = list(valid_for_item["store_id"].unique())

    fig, ax = plt.subplots(len(store_ids), 1, figsize=(8, 2 * len(store_ids)), sharex=True)
    colors = cm.viridis(np.linspace(0, 1, len(store_ids)))

    for i, store_id in enumerate(store_ids):
        valid_for_store = valid_for_item[valid_for_item["store_id"] == store_id]
        ax[i].plot(
            valid_for_store["date"].values,
            valid_for_store["count"].values,
#             label=id_store_map[store_id],
            color=colors[i]
        )
        ax[i].plot(
            valid_for_store["date"].values,
            valid_for_store["forecast"].values,
            color=colors[i],
            ls="--",
        )
#         ax[i].set(ylabel=f"Store {id_store_map[store_id]}")
    
#     ax[0].set(title=f"Item {id_item_map[item_id]}")
    for tick in ax[-1].get_xticklabels():
        tick.set_rotation(45)

    fig.tight_layout();


def aggregate_validation_plot(
    valid_df: pd.DataFrame,
    state: str,  #["CA", "TX", "WI"]
    cat: str, # ["FOODS", "HOBBIES", "HOUSEHOLD"]
):
    state_id = state_id_map[state]
    cat_id = cat_id_map[cat]
    
    state_cat_df = valid_df[(valid_df["state_id"] == state_id) & (valid_df["cat_id"] == cat_id)]
    g = state_cat_df.groupby(["store_id", "dept_id", "date"], observed=True)[["count", "forecast"]].sum()
    
    store_ids = g.index.get_level_values("store_id").unique()
    fig, ax = plt.subplots(len(store_ids), 1, figsize=(8, 2 * len(store_ids)), sharex=True)

    for i, store_id in enumerate(store_ids):
        ts = g.loc[store_id]

        dept_ids = ts.index.get_level_values("dept_id").unique()
        colors = cm.viridis(np.linspace(0, 1, len(dept_ids)))
        
        for j, dept_id in enumerate(dept_ids):
            ax[i].plot(
                ts.loc[dept_id].index,
                ts.loc[dept_id]["count"].values,
                label=id_dept_map[dept_id],
                color=colors[j]
            )
            ax[i].plot(
                ts.loc[dept_id].index,
                ts.loc[dept_id]["forecast"].values,
                color=colors[j],
                ls="--"
            )
#         ax[i].set(ylabel="count", title=id_store_map[store_id])
        ax[i].legend()

    for tick in ax[-1].get_xticklabels():
        tick.set_rotation(45)

    fig.tight_layout()

    
    
def residual_analysis():
    # Residuals for different items / stores / states / cats
    ...

In [ ]:
item_id = 0
item_level_validation_plot(valid_df, item_id)